使用autograd包来进行自动求梯度的有关操作

### 2.3.1 概念

+ 如果将Tensor的属性 .requires_grad 设置为True，它将开始追踪(track)在Tensor上的所有操作（这样就可以利用链式法则进行梯度传播了）。
+ 完成计算后，可以调用.backward()来完成所有梯度计算。
+ 此Tensor的梯度将累积到 .grad 属性中。

+ 如果不想被继续追踪，可以调用 .detach() 将其从追踪记录中分离出来，以防止此后的计算被追踪，这样梯度就传不过去了。
+ 还可以用 with torch.no_grad() 将不想被追踪的操作代码块包裹起来，此方法在评估模型的时候常用，因为在评估模型时不需要计算可训练参数（requires_grad=True）的梯度。

+ Tensor和Function互相结合可以构建一个记录有整个计算过程的有向无环图（DAG）。
+ 每个Tensor都有一个.grad_fn 属性，该属性就是创建该Tensor的Function，可判断该Tensor是不是通过某些运算得到的：若是，则grad_fn返回一个与这些运算相关的对象，若否，则返回None。

### 2.3.2 Tensor

In [1]:
# 创建一个Tensor并设置requires_grad=True
import torch
x = torch.ones(2, 2, requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [2]:
# 做运算后观察 结果变量的 .grad_fn 属性
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


因为x是直接创建的，所以它没有grad_fn，而y是通过一个加法操作创建的，所以它有一个为`<AddBackward0>`的grad_fn。

In [3]:
# 像x这种直接创建的称为叶子节点，叶子节点对应的grad_fn是None
print(x.is_leaf, y.is_leaf) # True False

True False


In [4]:
# 复杂运算
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [5]:
# 通过.requires_grad_()，来用in-place的方式改变requires_grad属性
a = torch.randn(2, 2) # 缺失情况下默认 requires_grad = False
a = ((a * 3) / (a - 1))
print(a.requires_grad) # False
a.requires_grad_(True)
print(a.requires_grad) # True
b = (a * a).sum()
print(b.grad_fn)

False
True


### 2.3.3 梯度

In [6]:
# out是一个标量，所以调用backward()时不需要指定求导变量
out.backward() # 等价于 out.backward(torch.tensor(1.))

In [7]:
# out关于x的梯度
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


+ out = (1/4)Σ3(xi + 2)²
+ 梯度 = out对x求偏导，xi = 1时，梯度为4.5

数学上，如果有一个函数值和自变量都为向量的函数y-> = f(x->)，那么y->关于x->的梯度就是一个雅可比矩阵（Jacobian matrix）
包 torch.autograd 可用于计算一些雅克比矩阵的乘积

grad在反向传播过程中是累加的(accumulated)，这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以一般在反向传播之前需把梯度清零

In [9]:
# 再反向传播一次，grad是累加的
# 调用.backward()来完成所有梯度计算
out2 = x.sum()
print(out2)
out2.backward()
print(x.grad)
out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor(4., grad_fn=<SumBackward0>)
tensor([[2., 2.],
        [2., 2.]])
tensor([[1., 1.],
        [1., 1.]])


+ 不允许张量对张量求导，只允许标量对张量求导，求导结果是和自变量同形的张量
+ 所以必要时要把张量通过将所有张量的元素加权求和的方式转换为标量
+ 例：设y由自变量x计算而来，w是和y同形的张量，则y.backward(w)的含义是：先计算t = torch.sum(y * w)，则t是个标量，然后求t对自变量x的导数

In [10]:
x = torch.tensor([1.0, 2.0, 3.0, 4.0], requires_grad=True)
y = 2 * x
z = y.view(2, 2)
print(z)

tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward0>)


In [11]:
# 现在 z 不是一个标量，所以在调用backward时需要传入一个和z同形的权重向量进行加权求和得到一个标量
v = torch.tensor([[1.0, 0.1], [0.01, 0.001]], dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])


> note：x.grad是和x同形的张量。

+ 中断梯度追踪的例子

In [12]:
x = torch.tensor(1.0, requires_grad=True)
y1 = x ** 2 
# 用with torch.no_grad() 将不想被追踪的操作代码块包裹起来
with torch.no_grad():
    y2 = x ** 3
y3 = y1 + y2
print(x.requires_grad)
print(y1, y1.requires_grad) # True
print(y2, y2.requires_grad) # False
print(y3, y3.requires_grad) # True

True
tensor(1., grad_fn=<PowBackward0>) True
tensor(1.) False
tensor(2., grad_fn=<AddBackward0>) True


In [13]:
# y3对x求梯度
y3.backward()
print(x.grad)

tensor(2.)


由于y2的定义被torch.no_grad():包裹，所以与y2有关的梯度不会回传，只有与y1有关的梯度才会回传，即y1=x^2对x的梯度

In [14]:
# 如果想修改tensor的数值，但是又不希望被autograd记录（不去影响反向传播），则可以对tensor.data进行操作
x = torch.ones(1,requires_grad=True)
print(x.data) # 还是一个tensor
print(x.data.requires_grad) # 但是已经是独立于计算图之外
y = 2 * x
x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播
y.backward()
print(x) # 更改data的值也会影响tensor的值
print(x.grad)

tensor([1.])
False
tensor([100.], requires_grad=True)
tensor([2.])
